In [414]:
import pandas as pd
import numpy as np

In [415]:
df = pd.read_csv('calidad-aire.csv')

df

,FECHA,HORA,CO_CENTENARIO,NO2_CENTENARIO,PM10_CENTENARIO,CO_CORDOBA,NO2_CORDOBA,PM10_CORDOBA,CO_LA_BOCA,NO2_LA_BOCA,PM10_LA_BOCA,CO_PALERMO,NO2_PALERMO,PM10_PALERMO
0,21NOV2009:00:00:00,8,0.39,17,NaN,s/d,20,NaN,s/d,18,NaN,0.56,NaN,NaN
1,21NOV2009:00:00:00,15,0.68,15,NaN,s/d,20,NaN,s/d,16,NaN,1.04,NaN,NaN
2,21NOV2009:00:00:00,21,0.81,20,NaN,s/d,18,NaN,s/d,23,NaN,1.16,NaN,NaN
3,21NOV2009:00:00:00,22,0.85,21,NaN,s/d,18,NaN,s/d,18,NaN,1.16,NaN,NaN
4,21NOV2009:00:00:00,24,0.93,22,NaN,s/d,15,NaN,s/d,25,NaN,1.16,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113622,26NOV2009:00:00:00,9,0.69,41,NaN,s/d,s/d,NaN,s/d,19,NaN,0.41,NaN,NaN
113623,18MAY2019:00:00:00,3,0.56,7,17,0.51,8,20,0.24,6,32,NaN,NaN,NaN
113624,03DEC2014:00:00:00,9,0.68,20,35,0.39,22,s/d,s/d,s/d,s/d,NaN,NaN,NaN
113625,03FEB2019:00:00:00,8,0.13,6,33,0.30,33,29,0.28,3,s/d,NaN,NaN,NaN


In [416]:
df['FECHA'].value_counts()

31MAY2019:00:00:00    49
01JUN2019:00:00:00    48
21DEC2021:00:00:00    41
20APR2012:00:00:00    38
22DEC2021:00:00:00    38
                      ..
25MAY2020:00:00:00    12
12FEB2010:00:00:00    12
17SEP2021:00:00:00    12
01OCT2009:00:00:00    11
01FEB2016:00:00:00     1
Name: FECHA, Length: 4770, dtype: int64

In [417]:
df.isnull().sum()

FECHA                   0
HORA                    0
CO_CENTENARIO         242
NO2_CENTENARIO        241
PM10_CENTENARIO     10548
CO_CORDOBA            242
NO2_CORDOBA           241
PM10_CORDOBA        10548
CO_LA_BOCA            239
NO2_LA_BOCA           241
PM10_LA_BOCA        10545
CO_PALERMO         107623
NO2_PALERMO        113627
PM10_PALERMO       113627
dtype: int64

In [418]:
df[df.duplicated()]

,FECHA,HORA,CO_CENTENARIO,NO2_CENTENARIO,PM10_CENTENARIO,CO_CORDOBA,NO2_CORDOBA,PM10_CORDOBA,CO_LA_BOCA,NO2_LA_BOCA,PM10_LA_BOCA,CO_PALERMO,NO2_PALERMO,PM10_PALERMO


In [419]:
df.dtypes

FECHA               object
HORA                 int64
CO_CENTENARIO       object
NO2_CENTENARIO      object
PM10_CENTENARIO     object
CO_CORDOBA          object
NO2_CORDOBA         object
PM10_CORDOBA        object
CO_LA_BOCA          object
NO2_LA_BOCA         object
PM10_LA_BOCA        object
CO_PALERMO          object
NO2_PALERMO        float64
PM10_PALERMO       float64
dtype: object

In [420]:
df['fecha_prep'] = df['FECHA'].str[:9]

df['fecha_prep']

0         21NOV2009
1         21NOV2009
2         21NOV2009
3         21NOV2009
4         21NOV2009
            ...    
113622    26NOV2009
113623    18MAY2019
113624    03DEC2014
113625    03FEB2019
113626    10OCT2018
Name: fecha_prep, Length: 113627, dtype: object

In [421]:
df['year'] = df['fecha_prep'].str[5:]

df['year']

0         2009
1         2009
2         2009
3         2009
4         2009
          ... 
113622    2009
113623    2019
113624    2014
113625    2019
113626    2018
Name: year, Length: 113627, dtype: object

In [422]:
df['mes']= df['fecha_prep'].str[2:5]

df['mes']

0         NOV
1         NOV
2         NOV
3         NOV
4         NOV
         ... 
113622    NOV
113623    MAY
113624    DEC
113625    FEB
113626    OCT
Name: mes, Length: 113627, dtype: object

In [423]:
df['dia'] = df['fecha_prep'].str[:2]

df['dia']

0         21
1         21
2         21
3         21
4         21
          ..
113622    26
113623    18
113624    03
113625    03
113626    10
Name: dia, Length: 113627, dtype: object

In [424]:
df_centenario = df[['FECHA','CO_CENTENARIO','NO2_CENTENARIO']]
#Columns with null values
df_centenario.isnull().sum()

FECHA               0
CO_CENTENARIO     242
NO2_CENTENARIO    241
dtype: int64

In [425]:
df_centenario.shape

(113627, 3)

In [426]:
#Total of null values
df_centenario.isnull().sum().sum()

483

In [427]:
df_centenario[df_centenario.isnull().any(axis=1)]

,FECHA,CO_CENTENARIO,NO2_CENTENARIO
271,18JAN2010:00:00:00,NaN,NaN
1031,19JAN2011:00:00:00,NaN,NaN
1313,09OCT2010:00:00:00,NaN,NaN
2277,19APR2012:00:00:00,NaN,NaN
2278,19APR2012:00:00:00,NaN,NaN
...,...,...,...
112099,01OCT2018:00:00:00,NaN,NaN
113060,31MAY2019:00:00:00,NaN,NaN
113436,01JUN2019:00:00:00,NaN,NaN
113528,29JAN2020:00:00:00,NaN,NaN


In [428]:
index =df_centenario[df_centenario.isnull().any(axis=1)].index

In [429]:
df_centenario.drop(index)

,FECHA,CO_CENTENARIO,NO2_CENTENARIO
0,21NOV2009:00:00:00,0.39,17
1,21NOV2009:00:00:00,0.68,15
2,21NOV2009:00:00:00,0.81,20
3,21NOV2009:00:00:00,0.85,21
4,21NOV2009:00:00:00,0.93,22
...,...,...,...
113622,26NOV2009:00:00:00,0.69,41
113623,18MAY2019:00:00:00,0.56,7
113624,03DEC2014:00:00:00,0.68,20
113625,03FEB2019:00:00:00,0.13,6


In [430]:
#df_centenario['fecha_'] =df_centenario['FECHA'].str[:9]

df_centenario.loc[:,'Fecha'] = df_centenario['FECHA'].str[:9]

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [431]:
df_centenario

,FECHA,CO_CENTENARIO,NO2_CENTENARIO,Fecha
0,21NOV2009:00:00:00,0.39,17,21NOV2009
1,21NOV2009:00:00:00,0.68,15,21NOV2009
2,21NOV2009:00:00:00,0.81,20,21NOV2009
3,21NOV2009:00:00:00,0.85,21,21NOV2009
4,21NOV2009:00:00:00,0.93,22,21NOV2009
...,...,...,...,...
113622,26NOV2009:00:00:00,0.69,41,26NOV2009
113623,18MAY2019:00:00:00,0.56,7,18MAY2019
113624,03DEC2014:00:00:00,0.68,20,03DEC2014
113625,03FEB2019:00:00:00,0.13,6,03FEB2019


In [432]:
df_centenario.dtypes

FECHA             object
CO_CENTENARIO     object
NO2_CENTENARIO    object
Fecha             object
dtype: object

In [433]:
df_centenario['CO_CENTENARIO'].value_counts()

s/d      27373
0.40      2109
0.41      2054
0.43      2054
0.44      2051
         ...  
2.36         1
5.04         1
5.14         1
4.68         1
22.38        1
Name: CO_CENTENARIO, Length: 367, dtype: int64

In [434]:
df_centenario['NO2_CENTENARIO'].value_counts()

s/d      21462
17        4212
14        4206
13        4175
15        4160
         ...  
8.00         1
43.00        1
155          1
19.00        1
9.00         1
Name: NO2_CENTENARIO, Length: 119, dtype: int64

In [435]:
code = {"s/d": 0}
#df_centenario['NO2_CENTENARIO'] = df_centenario['NO2_CENTENARIO'].map(code)
#df_centenario['CO_CENTENARIO'] = df_centenario['CO_CENTENARIO'].map(code)

df_centenario['CO_CENTENARIO']=df_centenario['CO_CENTENARIO'].replace("s/d",0)
df_centenario['NO2_CENTENARIO']=df_centenario['NO2_CENTENARIO'].replace("s/d",0)



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8504\3884878313.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_centenario['CO_CENTENARIO']=df_centenario['CO_CENTENARIO'].replace("s/d",0)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8504\3884878313.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_centenario['NO2_CENTENARIO']=df_centenario['NO2_CENTENARIO'].replace("s/d",0)


In [436]:
df_centenario['CO_CENTENARIO'].value_counts()

0        27373
0.40      2109
0.41      2054
0.43      2054
0.44      2051
         ...  
2.36         1
5.04         1
5.14         1
4.68         1
22.38        1
Name: CO_CENTENARIO, Length: 367, dtype: int64

In [437]:
df_centenario['NO2_CENTENARIO'].value_counts()

0        21462
17        4212
14        4206
13        4175
15        4160
         ...  
8.00         1
43.00        1
155          1
19.00        1
9.00         1
Name: NO2_CENTENARIO, Length: 119, dtype: int64

In [438]:
import re

In [439]:
re.match(r'^([\s\d]+)$', df_centenario['CO_CENTENARIO'])

TypeError: expected string or bytes-like object

In [ ]:
df_centenario["CO_CENTENARIO"]= df_centenario['CO_CENTENARIO'].str.replace("<", "")

#df_centenario["CO_CENTENARIO"]= df_centenario['CO_CENTENARIO'].str.isdigit()

df_centenario['CO_CENTENARIO'].value_counts()

AttributeError: Can only use .str accessor with string values!

In [ ]:
df_centenario['CO_CENTENARIO'] = df_centenario['CO_CENTENARIO'].astype(float)
df_centenario['NO2_CENTENARIO']= df_centenario['NO2_CENTENARIO'].astype(float)


ValueError: could not convert string to float: '#REF!'

In [ ]:
df_centenario['CO_CENTENARIO'].value_counts()

0.0    27373
Name: CO_CENTENARIO, dtype: int64

In [ ]:
df_centenario['NO2_CENTENARIO'].value_counts()

0.0      21523
17.0      4213
14.0      4207
13.0      4175
15.0      4161
         ...  
86.0         2
155.0        1
112.0        1
95.0         1
98.0         1
Name: NO2_CENTENARIO, Length: 92, dtype: int64

In [ ]:
df_centenario.groupby(['Fecha'])[['NO2_CENTENARIO','CO_CENTENARIO']].mean().reset_index()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8504\817832473.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_centenario.groupby(['Fecha'])[('NO2_CENTENARIO','CO_CENTENARIO')].mean().reset_index()
C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha
0,01APR2010
1,01APR2011
2,01APR2012
3,01APR2013
4,01APR2014
...,...
4765,31OCT2018
4766,31OCT2019
4767,31OCT2020
4768,31OCT2021


In [ ]:
df_centenario['CO_CENTENARIO'].value_counts()

0.0    27373
Name: CO_CENTENARIO, dtype: int64